In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Point Managers
todo

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P27.jpg" style="width: 200px; display: inline;">


## HasPts Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D50 HasPts Expanded.jpg" style="width: 200px; display: inline;">


### HasPts Members and Methods

## PLine Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D56 Pline Large.jpg" style="width: 800px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P16.jpg" style="width: 200px; display: inline;">


# Accessing and Manipulating HasPts Objects
todo

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P01.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P02.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P03.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P04.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P05.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P06.jpg" style="width: 200px; display: inline;">


# PGon and RGon Objects in Decod.es
todo

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.C01.jpg" style="width: 800px; display: inline;">


### PGons in Decod.es

### RGons in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P17.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P18.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P19.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P20.jpg" style="width: 200px; display: inline;">
